In [39]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [95]:
# Ensure data is processed
if not os.path.exists("../data/X.npy") or not os.path.exists("../data/Y.npy"):
    print("Processed data not found. Running VAE_dataprocessing.py...")
    os.system("python ../VAE_dataprocessing.py")

# Load processed training data
X = np.load("../data/X.npy")
Y = np.load("../data/Y.npy")

# Train and Validation Sets
train_X = X[:39]  # shape: (39, 3, 24)
val_X = X[39:]    # shape: (10, 3, 24)
train_Y = Y[:39]  # shape: (39, 24)
val_Y = Y[39:]    # shape: (10, 24)

# Normalize X
X_mean = np.mean(train_X, axis=0, keepdims=True)
X_std = np.std(train_X, axis=0, keepdims=True)

train_X_norm = (train_X - X_mean) / X_std
val_X_norm   = (val_X - X_mean) / X_std

print(train_X_norm[:5])

# Normalize Y
Y_mean = np.mean(train_Y, axis=0, keepdims=True)
Y_std = np.std(train_Y, axis=0, keepdims=True)

# Normalize both training and validation Y using training set statistics
train_Y_norm = (train_Y - Y_mean) / Y_std
val_Y_norm   = (val_Y - Y_mean) / Y_std


[[[ 1.96544691  1.89090018  1.63103472  1.55475405  1.57971671
    1.47559902  1.54613365  1.20877907  1.2023891   1.16062725
    0.90683138  0.36975428  0.21047131  0.50433129  0.66391452
    0.93693405  0.9550858   0.78086046  0.50445886  0.2969833
   -0.14215755 -0.38787225 -0.33649858 -0.10276504]
  [ 2.21877628  2.28797191  2.36326372  2.38857246  2.4282269
    2.39361263  2.38429869  2.37805714  2.29728787  2.19367929
    2.04724362  1.87403809  1.65201661  1.46909818  1.32787373
    1.35979494  1.29665935  1.13896049  0.92962441  0.76941787
    0.56851036  0.41031165  0.33490702  0.10594362]
  [ 2.48682813  2.38709946  2.33682649  2.19033722  2.13671521
    2.07027948  2.07932324  2.02947926  2.0293772   2.11176924
    2.16165408  2.26257333  2.34405927  2.2456411   2.09718512
    1.81197206  1.53871871  1.49888962  1.31611648  1.13080152
    0.84082187  0.61034226  0.50157048  0.39697204]]

 [[-0.03159875  0.09864043  0.48194915  0.44053271  0.01202576
   -0.15674968 -0.1650566

In [84]:
train_X_tensor = torch.tensor(train_X_norm, dtype=torch.float32)
train_Y_tensor = torch.tensor(train_Y_norm, dtype=torch.float32)
val_X_tensor   = torch.tensor(val_X_norm, dtype=torch.float32)
val_Y_tensor   = torch.tensor(val_Y_norm, dtype=torch.float32)

train_dataset = TensorDataset(train_X_tensor, train_Y_tensor)
val_dataset = TensorDataset(val_X_tensor, val_Y_tensor)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [85]:
# Model Definitions
latent_dim = 16
x_dim = 3 * 24
y_dim = 24

In [86]:
class Encoder(nn.Module):
    def __init__(self, x_dim, y_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(x_dim + y_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc_mu = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)
    
    def forward(self, x, y):
        combined = torch.cat([x, y], dim=1)
        h = F.relu(self.fc1(combined))
        h = F.relu(self.fc2(h))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

In [87]:
class Decoder(nn.Module):
    def __init__(self, x_dim, latent_dim, y_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(x_dim + latent_dim, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, y_dim)
    
    def forward(self, x, z):
        combined = torch.cat([x, z], dim=1)
        h = F.relu(self.fc1(combined))
        h = F.relu(self.fc2(h))
        y_recon = self.fc3(h)
        return y_recon

In [88]:
# CVAE model: combines encoder, reparameterization, and decoder
class CVAE(nn.Module):
    def __init__(self, x_dim, y_dim, latent_dim):
        super(CVAE, self).__init__()
        self.encoder = Encoder(x_dim, y_dim, latent_dim)
        self.decoder = Decoder(x_dim, latent_dim, y_dim)
    
    def reparameterize(self, mu, logvar):
        logvar = torch.clamp(logvar, min=-10, max=10)
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x, y):
        # In training, y is provided to the encoder
        mu, logvar = self.encoder(x, y)
        z = self.reparameterize(mu, logvar)
        y_recon = self.decoder(x, z)
        return y_recon, mu, logvar

In [89]:
def loss_function(y_recon, y, mu, logvar):
    recon_loss = F.mse_loss(y_recon, y, reduction='mean')
    kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss

In [90]:
model = CVAE(x_dim, y_dim, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 100

In [91]:
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0
    for batch_x, batch_y in train_loader:
        # Flatten X from (batch, 3, 24) to (batch, 72)
        batch_x = batch_x.view(batch_x.size(0), -1)
        optimizer.zero_grad()
        y_recon, mu, logvar = model(batch_x, batch_y)
        loss = loss_function(y_recon, batch_y, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_x.size(0)
    train_loss /= len(train_dataset)
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.view(batch_x.size(0), -1)
            y_recon, mu, logvar = model(batch_x, batch_y)
            loss = loss_function(y_recon, batch_y, mu, logvar)
            val_loss += loss.item() * batch_x.size(0)
    val_loss /= len(val_dataset)
    
    print(f"Epoch {epoch}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


Epoch 1/100, Train Loss: 1.0120, Val Loss: 0.8278
Epoch 2/100, Train Loss: 0.9829, Val Loss: 0.8157
Epoch 3/100, Train Loss: 0.9633, Val Loss: 0.8063
Epoch 4/100, Train Loss: 0.9337, Val Loss: 0.8054
Epoch 5/100, Train Loss: 0.9140, Val Loss: 0.8275
Epoch 6/100, Train Loss: 0.8891, Val Loss: 0.8430
Epoch 7/100, Train Loss: 0.8703, Val Loss: 0.8560
Epoch 8/100, Train Loss: 0.8557, Val Loss: 0.9243
Epoch 9/100, Train Loss: 0.8317, Val Loss: 0.9562
Epoch 10/100, Train Loss: 0.8089, Val Loss: 0.9932
Epoch 11/100, Train Loss: 0.7757, Val Loss: 1.0405
Epoch 12/100, Train Loss: 0.7593, Val Loss: 1.0852
Epoch 13/100, Train Loss: 0.7509, Val Loss: 1.0994
Epoch 14/100, Train Loss: 0.7236, Val Loss: 1.0999
Epoch 15/100, Train Loss: 0.6899, Val Loss: 1.1745
Epoch 16/100, Train Loss: 0.6636, Val Loss: 1.1816
Epoch 17/100, Train Loss: 0.6487, Val Loss: 1.1837
Epoch 18/100, Train Loss: 0.6181, Val Loss: 1.2231
Epoch 19/100, Train Loss: 0.5741, Val Loss: 1.2443
Epoch 20/100, Train Loss: 0.5749, Val Lo

In [92]:
model.eval()
with torch.no_grad():
    sample_x, sample_y = next(iter(val_loader))
    sample_x = sample_x.view(sample_x.size(0), -1)
    y_recon, _, _ = model(sample_x, sample_y)
    print("\nValidation Sample:")
    print("True Y (first sample):")
    print(sample_y[0])
    print("Predicted Y (first sample):")
    print(y_recon[0])


Validation Sample:
True Y (first sample):
tensor([-0.1349,  0.1399, -0.1809,  0.4955,  0.6899,  0.6104,  0.7086,  0.4167,
         0.5230, -0.7771, -0.9032, -0.8574,  0.3308, -0.4826, -0.6513, -0.4828,
        -0.5125,  0.1398,  0.4041, -0.5161,  0.6285,  0.0626, -0.0558, -0.7194])
Predicted Y (first sample):
tensor([-0.4228, -0.4991, -0.3475, -0.2164, -0.3202, -0.0194, -0.8071, -0.1626,
         0.1505, -0.1739, -0.4011, -0.5874, -0.4829, -0.6422, -0.6498, -0.9500,
        -0.7930,  0.1489,  0.0173,  0.1896,  0.0671, -0.0360, -0.0856, -0.9487])
